### Generate features from Fourier coefficients to use as train data

In [1]:
# Libraries import
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import math
import os
from os import listdir
from os.path import isfile, join

%matplotlib inline
# Get current/root directory
root = os.getcwd()

In [2]:
# Plot style
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (18, 8)

In [3]:
data = []
sig_arr = []
pid = 1
N = 256
# compute--
# magnitude   phase   signature   distance   sector

#### Feature 1: Compute spectral rotation angle given r, c

<img height="400" width="400" src="images/f1.PNG" />

In [ ]:
# Compute spectral rotation angle given r,c
def spectral_rotation_angle(r, c):
    x = r - (N / 2)
    y = (N / 2) - c
    q = 0 if x == 0 or y == 0 else (y / x)
    phase = math.atan(q)
    return phase

#### Feature 2: Compute distance given r, c

In [ ]:
# Compute distance given r,c
def compute_distance(r, c):
    x = r - (N / 2)
    y = (N / 2) - c
    d = math.sqrt(x*x+y*y)
    return d

#### Feature 3: Compute sector index

<img height="400" width="400" src="images/f2.PNG" />

In [3]:
# Compute sector index
def compute_sector(phase):
    phase_d = abs(np.degrees(phase))
    #print(phase_d)
    v = (phase_d/22.5)//1 #get only integer
    if v == 16:
        return 0
    return v

#### Feature 4: Computing spectral deviation angle

<img height="400" width="400" src="images/f3.PNG" />

In [4]:
# Computing spectral deviation angle
def compute_theta(p1, p2, p3):
    # p1(x1,y1)
    # p2(x2,y2)
    # p3(x3,y3)
    HX1 = []
    HX2 = []
    
    HX1.append(p1[0]-5) # x1 0
    HX1.append(p1[1]) # y1 1
    HX1.append(p1[0]+5) # x2 2
    HX1.append(p1[1]) # y2 3
    
    HX2.append(p2[0]-5)
    HX2.append(p2[1])
    HX2.append(p2[0]+5)
    HX2.append(p2[1])
    
    M1 = (HX1[3]-HX1[1])/(HX1[2]-HX1[0]) # Slope of HX1
    M2 = (p2[1]-p1[1])/(p2[0]-p1[0]) # Slope of p1p2
    M3 = (HX2[3]-HX2[1])/(HX2[2]-HX2[0]) # Slope of HX2
    M4 = (p3[1]-p2[1])/(p3[0]-p2[0]) # Slope of p2p3
    
    TAN1 = abs((M2-M1)/(1+M1*M2))
    THETA1 = math.atan(TAN1)
    
    TAN2 = abs((M4-M3)/(1+M3*M4))
    THETA2 = math.atan(TAN2)
    
    # For negative slope, THETA1 is negative
    if M2 < 0:
        THETA1 = -THETA1
        
    if p3[1] < p2[1]:
        THETA2 = -THETA2
    
    THETA3 = THETA2-THETA1
    
    #OBJ.append(THETA3)
    
    return THETA3

#### Use the eq/functions above to generate features

In [4]:
loc = 'D:/SCA/256_256/txt'
files = [f for f in listdir(loc) if isfile(join(loc, f))]

for file in files:
    print(file)
    
    arr = file.split('_')
    sl_no = arr[0]
    slice_no = arr[1]
    subslice_no= arr[2]
    otype = arr[3]
    x_0 = arr[4]
    y_0 = arr[4].split('.')[0]
    
    dataset=pd.read_csv(loc + "/" + file, header=None, delimiter="\t")   
    dataset = dataset.values
    
    for ds in dataset:
        temp = []
        st = ds[0].replace('<', '')
        st = st.replace('>', '')
        d = st.split(',')
        d = np.array(d, dtype=np.float32)

        spectrum = d[2]
        phase = spectral_rotation_angle(d[0], d[1])
        distance = compute_distance(d[0], d[1])
        sector = compute_sector(phase)
        signature_angle = 0
        
        if len(sig_arr) < 3:
            p = []
            x = d[0]-(N / 2) #r - (N / 2)
            y = (N / 2) - d[1] #(N / 2) - c
            p.append(x)
            p.append(y)
            sig_arr.append(p)
            
            if len(sig_arr) == 3:
                THETA = compute_theta(sig_arr[0], sig_arr[1], sig_arr[2])
                signature_angle = THETA
            else:
                signature_angle = 0
        else:
            p = []
            x = d[0]-(N / 2) #r - (N / 2)
            y = (N / 2) - d[1] #(N / 2) - c
            
            p.append(x)
            p.append(y)
            sig_arr[0] = sig_arr[1]
            sig_arr[1] = sig_arr[2]
            sig_arr[2] = p
            
            THETA = compute_theta(sig_arr[0], sig_arr[1], sig_arr[2])
            signature_angle = THETA
        
        if math.isnan(spectrum):
            spectrum = 0
            
        if math.isnan(phase):
            phase = 0
            
        if math.isnan(sector):
            sector = 0
            
        if math.isnan(distance):
            distance = 0
        
        if math.isnan(signature_angle):
            signature_angle = 0
            
        # mag,phase,signature,sector,distance
        str_data = str(spectrum)+','+str(phase)+','+str(signature_angle)+','+str(sector)+','+str(distance)
        
        file_object = open('D:/SCA/256_256/feature_map/test_unet/'+file, 'a+')
        file_object.write(str_data+'\n')
        file_object.close()

102_114_1_e_h_261_153.txt
102_115_1_c_248_226.txt
102_115_1_c_271_244.txt
...
...
